In [1]:
import csv
import requests
import re
import time
import itertools
import pandas as pd

In [57]:
#Gets a protein sequence in FASTA format given the ensembl transcript ID
def get_protein_sequence(canonical_transcript):
    server = "http://grch37.rest.ensembl.org"
    ext = "/sequence/id/" + canonical_transcript + "?content-type=text/x-fasta;type=protein"
    r = requests.get(server + ext, headers={"Content-Type": "text/x-fasta"})
    time.sleep(3)
    
    if not r.ok:
        try:
            r.raise_for_status()
            return "error"
        except requests.exceptions.HTTPError: #I should catch the error too or print, to find out which specific genes
            pass
    seqlist = r.text.split("\n", 1)
    
    if len(seqlist) == 2: #A lot of these transcripts are noncoding or introns, don't know why
        sequence = seqlist[1]
        sequence = sequence.replace("\n", "")
        return sequence
    #print("Sequence found")

#Adapted from https://github.com/xjenny2/phospho-programs/blob/master/ensembl.py

In [20]:
tsv_file = open("filtered_mapRef.tsv")
mart_tsv = csv.reader(tsv_file, delimiter="\t", quotechar='"')
df = pd.DataFrame(mart_tsv)
dfRef = df[df[6].astype(bool)] #filters tsv file such that it only considers all with a RefSeq Transcript

In [21]:
dfRef.shape

(15543, 7)

In [2]:
tsv_file = open("batchJob.txt")
mart_tsv = csv.reader(tsv_file, delimiter="\t", quotechar='"')

In [3]:
df = pd.DataFrame(mart_tsv)

In [4]:
df.shape

(27867, 106)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,Input Variant,Errors,Chromosomal Variant,Coding Variant(s),None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,NC_000001.11:g.943995C>T,,NC_000001.11:g.943995C>T,NM_015658.3:c.*699G>A,NM_152486.2:c.1888C>T,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,NC_000001.11:g.944051T>G,,NC_000001.11:g.944051T>G,NM_015658.3:c.*643A>C,NM_152486.2:c.1944T>G,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,NC_000001.11:g.1014143C>T,,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,NC_000001.11:g.1014359G>T,,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df2 = df[[2,3]]

In [7]:
df2.rename(columns=df.iloc[0])

,Chromosomal Variant,Coding Variant(s)
0,Chromosomal Variant,Coding Variant(s)
1,NC_000001.11:g.943995C>T,NM_015658.3:c.*699G>A
2,NC_000001.11:g.944051T>G,NM_015658.3:c.*643A>C
3,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T
4,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T
5,NC_000001.11:g.1041582C>T,XM_006710635.2:c.1057C>T
6,NC_000001.11:g.1042053C>G,XM_006710635.2:c.1275C>G
7,NC_000001.11:g.1045487C>T,XM_006710635.2:c.2500C>T
8,NC_000001.11:g.1049672C>T,XM_006710635.2:c.4621C>T
9,NC_000001.11:g.1232517G>A,NM_080605.3:c.239G>A


In [11]:
df2 = df2[df2[3].str.contains("NM")]

In [13]:
df2 = df2[~df2[3].str.contains("-")]

In [16]:
df2 = df2[~df2[3].str.contains("\*")]

In [17]:
df2.shape

(8429, 2)

In [25]:
df2 = df2.rename(columns=df.iloc[0])

In [26]:
df2['ENST'] = ""

In [28]:
for index, row in df2.iterrows():
    NM = row[1].split('.')[0] #obtains the initial transcript
    for x, y in dfRef.iterrows():
        Match = y[6].split('.')[0] 
        if NM == Match: #checks a map for a transcript
            df2.loc[index, 'ENST'] = y[4]

In [30]:
nan_value = float("NaN")
df2.replace("", nan_value, inplace = True)
df2.dropna(subset = ['ENST'], inplace=True)

In [31]:
df2.shape

(4686, 4)

In [38]:
df2['ENST'] = df2['ENST'].str.replace("NaN ","")

In [39]:
df2

,Gene end (bp),Gene stable ID,,ENST
3,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,NaN,ENST00000649529
4,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,NaN,ENST00000649529
9,NC_000001.11:g.1232517G>A,NM_080605.3:c.239G>A,NaN,ENST00000379198
10,NC_000001.11:g.1232959C>G,NM_080605.3:c.681C>G,NaN,ENST00000379198
11,NC_000001.11:g.1233041C>T,NM_080605.3:c.763C>T,NaN,ENST00000379198
15,NC_000001.11:g.1518966C>T,NM_001170535.1:c.490C>T,NaN,ENST00000378756
16,NC_000001.11:g.1520284C>T,NM_001170535.1:c.658C>T,NaN,ENST00000378756
17,NC_000001.11:g.1535392G>C,NM_001114748.1:c.489C>G,NaN,ENST00000378733
19,NC_000001.11:g.2229502C>T,NM_003036.3:c.736C>T,NaN,ENST00000378536
20,NC_000001.11:g.2306202C>A,NM_003036.3:c.1950C>A,NaN,ENST00000378536


In [43]:
df2.dtypes

Gene end (bp)      object
Gene stable ID     object
                  float64
ENST               object
dtype: object

In [48]:
dfFinal = df2[['Gene end (bp)','Gene stable ID', 'ENST']]

In [53]:
dfFinal = dfFinal.rename(columns = {'Gene end (bp)':'Chromosomal Variant', 'Gene stable ID':'Coding Variant', 'ENST':'ENST'})

In [54]:
dfFinal

,Chromosomal Variant,Coding Variant,ENST
3,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,ENST00000649529
4,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,ENST00000649529
9,NC_000001.11:g.1232517G>A,NM_080605.3:c.239G>A,ENST00000379198
10,NC_000001.11:g.1232959C>G,NM_080605.3:c.681C>G,ENST00000379198
11,NC_000001.11:g.1233041C>T,NM_080605.3:c.763C>T,ENST00000379198
15,NC_000001.11:g.1518966C>T,NM_001170535.1:c.490C>T,ENST00000378756
16,NC_000001.11:g.1520284C>T,NM_001170535.1:c.658C>T,ENST00000378756
17,NC_000001.11:g.1535392G>C,NM_001114748.1:c.489C>G,ENST00000378733
19,NC_000001.11:g.2229502C>T,NM_003036.3:c.736C>T,ENST00000378536
20,NC_000001.11:g.2306202C>A,NM_003036.3:c.1950C>A,ENST00000378536


In [55]:
dfFinal['ProteinSeq'] = ""

In [56]:
dfFinal.shape

(4686, 4)

In [58]:
for index, row in dfFinal.iterrows():
    dfFinal.loc[index, 'ProteinSeq'] = get_protein_sequence(row[2])

In [59]:
dfFinal.dropna(subset = ['ProteinSeq'])

,Chromosomal Variant,Coding Variant,ENST,ProteinSeq
9,NC_000001.11:g.1232517G>A,NM_080605.3:c.239G>A,ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...
10,NC_000001.11:g.1232959C>G,NM_080605.3:c.681C>G,ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...
11,NC_000001.11:g.1233041C>T,NM_080605.3:c.763C>T,ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...
15,NC_000001.11:g.1518966C>T,NM_001170535.1:c.490C>T,ENST00000378756,MSWLFGINKGPKGEGAGPPPPLPPAQPGAEGGGDRGLGDRPAPKDK...
16,NC_000001.11:g.1520284C>T,NM_001170535.1:c.658C>T,ENST00000378756,MSWLFGINKGPKGEGAGPPPPLPPAQPGAEGGGDRGLGDRPAPKDK...
17,NC_000001.11:g.1535392G>C,NM_001114748.1:c.489C>G,ENST00000378733,MSMSANTMIFMILGASVVMAIACLMDMNALLDRFHNYILPHLRGED...
19,NC_000001.11:g.2229502C>T,NM_003036.3:c.736C>T,ENST00000378536,MEAAAGGRGCFQPHPGLQKTLEQFHLSSMSSLGGPAAFSARWAQEA...
20,NC_000001.11:g.2306202C>A,NM_003036.3:c.1950C>A,ENST00000378536,MEAAAGGRGCFQPHPGLQKTLEQFHLSSMSSLGGPAAFSARWAQEA...
21,NC_000001.11:g.2406561C>A,NM_002617.3:c.835G>T,ENST00000447513,MAPAAASPPEVIRAAQKDEYYRGGLRSAAGGALHSLAGARKWLEWR...
22,NC_000001.11:g.2406606C>A,NM_002617.3:c.790G>T,ENST00000447513,MAPAAASPPEVIRAAQKDEYYRGGLRSAAGGALHSLAGARKWLEWR...


In [60]:
dfFinal.to_csv('temp_final.csv')

In [61]:
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper
hdp = hgvs.dataproviders.uta.connect()
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name = 'GRCh38', alt_aln_method = "splign", replace_reference = True)

In [62]:
dfFinal['Protein Variant'] = ""

In [67]:
dfFinal = dfFinal[dfFinal['ENST'].astype(bool)]

In [72]:
dfFinal.to_csv('temp_final_3.csv')

In [79]:
df = dfFinal[dfFinal['ProteinSeq'].astype(str).str.startswith('M')]

In [85]:
df['Protein Variant'] = ""

/Users/dylantom/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
for index, row in df.iterrows():
    hgvs_g = str(row[0])
    hp = hgvs.parser.Parser()
    var_g = hp.parse_hgvs_variant(hgvs_g)
    transcripts = am.relevant_transcripts(var_g)
    if transcripts[0].startswith('NR'):
        continue
    else:
        var_c = am.g_to_c(var_g, transcripts[0]) #converts Gene to Transcript
        var_p = am.c_to_p(var_c) #converts Transcript to Protein

    df.loc[index, 'Protein Variant'] = str(var_p)

/Users/dylantom/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001114748.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_003036.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_153818.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?d

In [66]:
dfFinal.to_csv('temp_final_2.csv')